In [1]:
import numpy as np

from pycox import datasets
from lifelines.datasets import load_rossi
from sksurv.datasets import (
    load_aids,
    load_breast_cancer,
    load_flchain,
    load_gbsg2,
    load_whas500,
)
from sklearn.preprocessing import LabelEncoder


def get_dataset(name: str):
    if name == "metabric":
        df = datasets.metabric.read_df()
    elif name == "support":
        df = datasets.support.read_df()
    elif name == "gbsg":
        df = datasets.gbsg.read_df()
    elif name == "rossi":
        df = load_rossi()
        df = df.rename(columns={"week": "duration", "arrest": "event"})
    elif name == "aids":
        X, Y = load_aids()
        Y_unp = np.array(Y, dtype=[("event", "int"), ("duration", "float")])
        df = X.copy()
        df["event"] = Y_unp["event"]
        df["duration"] = Y_unp["duration"]
    elif name == "flchain":
        X, Y = load_flchain()
        Y_unp = np.array(Y, dtype=[("event", "int"), ("duration", "float")])
        df = X.copy()
        df["event"] = Y_unp["event"]
        df["duration"] = Y_unp["duration"]
    elif name == "gbsg2":
        X, Y = load_gbsg2()
        Y_unp = np.array(Y, dtype=[("event", "int"), ("duration", "float")])
        df = X.copy()
        df["event"] = Y_unp["event"]
        df["duration"] = Y_unp["duration"]
    elif name == "whas500":
        X, Y = load_whas500()
        Y_unp = np.array(Y, dtype=[("event", "int"), ("duration", "float")])
        df = X.copy()
        df["event"] = Y_unp["event"]
        df["duration"] = Y_unp["duration"]

    for col in df.columns:
        if df[col].dtype.name in ["object", "category"]:
            df[col] = LabelEncoder().fit_transform(df[col])

    duration_col = "duration"
    event_col = "event"

    X = df.drop(columns=[duration_col, event_col])
    Y = df[event_col]
    T = df[duration_col]

    time_horizons = np.linspace(T.min(), T.max(), num=5)[1:-1]

    return df, X, T, Y, time_horizons

In [2]:
from synthcity.plugins import Plugins
from adjutorium.utils.tester import evaluate_survival_estimator
from adjutorium.plugins.prediction.risk_estimation import RiskEstimation
import pandas as pd


def constant_columns(dataframe: pd.DataFrame) -> list:
    """
    Drops constant value columns of pandas dataframe.
    """
    result = []
    for column in dataframe.columns:
        if len(dataframe[column].unique()) == 1:
            result.append(column)
    return result


def _train_and_evaluate(
    X_train, T_train, Y_train, X_test, T_test, Y_test, time_horizons, n_folds=3
):
    predictor = RiskEstimation().get("survival_xgboost")
    n_folds = 3

    const_cols = constant_columns(X_train)
    X_train = X_train.drop(columns=const_cols)
    X_test = X_test.drop(columns=const_cols)
    try:
        predictor.fit(X_train, T_train, Y_train)

        return evaluate_survival_estimator(
            [predictor] * n_folds,
            X_test,
            T_test,
            Y_test,
            time_horizons=time_horizons,
            n_folds=n_folds,
            pretrained=True,
        )["str"]

    except BaseException:
        return {
            "c_index": "0 +/-0",
            "aucroc": "0 +/- 0",
            "brier_score": "1 +/- 0",
        }


def _fold_evaluate(
    X_test,
    T_test,
    Y_test,
    time_horizons,
    n_folds=3,
):
    predictor = RiskEstimation().get("survival_xgboost")
    n_folds = 3

    const_cols = constant_columns(X_test)
    X_test = X_test.drop(columns=const_cols)

    try:
        return evaluate_survival_estimator(
            predictor,
            X_test,
            T_test,
            Y_test,
            time_horizons=time_horizons,
            n_folds=n_folds,
        )["str"]
    except BaseException:
        return {
            "c_index": "0 +/-0",
            "aucroc": "0 +/- 0",
            "brier_score": "1 +/- 0",
        }


def evaluate_surv_generation(generative_method: str, dataset: str):
    df, X_real, T_real, Y_real, time_horizons = get_dataset(dataset)

    syn_model = Plugins().get(generative_method)
    syn_model.fit(df)

    df_generated = syn_model.generate()

    X_syn = df_generated.drop(columns=["duration", "event"])
    Y_syn = df_generated["event"]
    T_syn = df_generated["duration"]

    real_real_score = _fold_evaluate(X_real, T_real, Y_real, time_horizons)
    syn_syn_score = _fold_evaluate(X_syn, T_syn, Y_syn, time_horizons)
    real_syn_score = _train_and_evaluate(
        X_real, T_real, Y_real, X_syn, T_syn, Y_syn, time_horizons
    )
    syn_real_score = _train_and_evaluate(
        X_syn, T_syn, Y_syn, X_real, T_real, Y_real, time_horizons
    )

    return {
        "real_real": real_real_score,
        "syn_syn": syn_syn_score,
        "real_syn": real_syn_score,
        "syn_real": syn_real_score,
    }

In [3]:
from math import sqrt
from sklearn.metrics import mean_squared_error


def expected_time_error(T, E, predicted_event_time):
    censored_err = T[E == 0] - predicted_event_time[E == 0]
    censored_err[censored_err < 0] = 0

    return sqrt(
        mean_squared_error(T[E == 1], predicted_event_time[E == 1])
        + np.mean(censored_err ** 2)
    )


def ranking_error(T, E, predicted_time):
    rank_errs = 0

    for idx in range(len(T)):
        actual_order = (T[E == 1] <= T[idx]).astype(int).squeeze()
        pred_order = (
            (predicted_time[E == 1] <= predicted_time[idx]).astype(int).squeeze()
        )
        rank_errs += mean_squared_error(pred_order, actual_order)

    return rank_errs

## AIDS dataset

In [4]:
import pandas as pd

df, X_real, T_real, Y_real, time_horizons = get_dataset("aids")

df

,age,cd4,hemophil,ivdrug,karnof,priorzdv,raceth,sex,strat2,tx,txgrp,event,duration
0,34.0,169.0,0,0,0,39.0,0,0,1,0,0,0,189.0
1,34.0,149.5,0,0,3,15.0,1,1,1,0,0,0,287.0
2,20.0,23.5,1,0,0,9.0,0,0,0,1,1,0,242.0
3,48.0,46.0,0,0,3,53.0,0,0,1,0,0,0,199.0
4,46.0,10.0,0,2,3,12.0,0,0,0,1,1,0,286.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,44.0,65.5,0,0,0,103.0,0,0,1,1,1,0,273.0
1147,41.0,7.5,0,0,2,20.0,1,0,0,1,1,1,47.0
1148,43.0,170.0,0,2,3,27.0,1,0,1,0,0,0,272.0
1149,44.0,282.5,0,2,2,12.0,0,0,1,0,0,0,192.0


In [5]:
df[df["event"] == 1]

,age,cd4,hemophil,ivdrug,karnof,priorzdv,raceth,sex,strat2,tx,txgrp,event,duration
13,33.0,16.0,0,0,2,24.0,1,1,0,0,0,1,206.0
16,35.0,17.5,0,0,3,40.0,2,0,0,0,0,1,298.0
22,33.0,23.0,0,0,3,6.0,2,0,0,0,0,1,190.0
29,31.0,11.0,0,0,3,12.0,1,0,0,0,0,1,82.0
56,47.0,33.5,0,0,2,28.0,2,0,0,0,0,1,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,42.0,35.5,0,0,3,32.0,0,0,0,0,0,1,186.0
1106,56.0,63.0,0,0,2,48.0,1,0,1,0,0,1,114.0
1120,45.0,64.0,0,0,2,30.0,0,1,1,0,0,1,20.0
1126,39.0,18.0,0,0,2,22.0,2,0,0,1,1,1,174.0


# COX PH expectation

In [222]:
from lifelines import CoxPHFitter

model = CoxPHFitter()

model.fit(df, "duration", "event")

<lifelines.CoxPHFitter: fitted with 1151 total observations, 1055 right-censored observations>

In [223]:
# Expectation using trapezoidal rule
from scipy.integrate import trapz

surv_f = model.predict_survival_function(df)

coxph_expectation = pd.Series(
    trapz(surv_f.values.T, surv_f.index), index=surv_f.T.index
)

coxph_expectation

0       359.003910
1       354.991700
2       349.654394
3       314.187092
4       334.745364
           ...    
1146    349.480252
1147    322.003981
1148    358.798987
1149    362.359435
1150    355.562560
Length: 1151, dtype: float64

In [224]:
expected_time_error(df["duration"], df["event"], coxph_expectation)

228.63227491698547

In [225]:
ranking_error(df["duration"], df["event"], coxph_expectation)

362.12499999999983

In [227]:
(T <= coxph_expectation).sum() / len(coxph_expectation)

0.9287576020851434

## Random survival forest

In [228]:
from sksurv.ensemble import RandomSurvivalForest

model = RandomSurvivalForest()

X = df.drop(columns=["event", "duration"])
y = df[["event", "duration"]]

y = [(df["event"].iloc[i], df["duration"].iloc[i]) for i in range(len(y))]
y = np.array(y, dtype=[("status", "bool"), ("time", "<f8")])

model.fit(X.values, y)

RandomSurvivalForest()

In [229]:
expectations = []
for surv_f in model.predict_survival_function(X):
    expectations.append(trapz(surv_f.y, surv_f.x))

rsf_expected_time = pd.Series(expectations, index=X.index)

rsf_expected_time

0       288.776432
1       296.054711
2       271.956147
3       279.186137
4       273.784811
           ...    
1146    292.560137
1147    198.523644
1148    295.494869
1149    294.069708
1150    296.988556
Length: 1151, dtype: float64

In [230]:
expected_time_error(df["duration"], df["event"], rsf_expected_time)

136.85045142871962

In [231]:
ranking_error(df["duration"], df["event"], rsf_expected_time)

169.7187499999999

In [232]:
(T <= rsf_expected_time).sum() / len(rsf_expected_time)

0.6629018245004344

## Survival XGBoost

In [233]:
from xgbse.converters import convert_to_structured

X = df.drop(columns=["event", "duration"])
E = df["event"]
T = df["duration"]

y = convert_to_structured(T, E)

In [234]:
from xgbse import XGBSEDebiasedBCE

# fitting xgbse model
xgbse_model = XGBSEDebiasedBCE()
xgbse_model.fit(X, y)

XGBSEDebiasedBCE(lr_params={'C': 0.001, 'max_iter': 500},
                 xgb_params={'aft_loss_distribution': 'normal',
                             'aft_loss_distribution_scale': 1,
                             'booster': 'dart', 'colsample_bynode': 0.5,
                             'eval_metric': 'aft-nloglik',
                             'learning_rate': 0.05, 'max_depth': 8,
                             'min_child_weight': 50,
                             'objective': 'survival:aft', 'subsample': 0.5,
                             'tree_method': 'hist'})

In [235]:
surv_f = xgbse_model.predict(X)

xgb_expected_time = pd.Series(trapz(surv_f.values, surv_f.T.index), index=surv_f.index)

xgb_expected_time

0       281.952605
1       284.301821
2       276.389738
3       274.693388
4       265.944473
           ...    
1146    281.949675
1147    260.382612
1148    281.461917
1149    280.725644
1150    283.447342
Length: 1151, dtype: float64

In [236]:
expected_time_error(T, E, xgb_expected_time)

183.16253863127292

In [237]:
ranking_error(df["duration"], df["event"], xgb_expected_time)

347.43750000000034

In [238]:
(T <= xgb_expected_time).sum() / len(xgb_expected_time)

0.6177237185056472

## Deephit

In [239]:
from adjutorium.plugins.prediction.risk_estimation import RiskEstimation

model = RiskEstimation().get("deephit", n_iter=5000)

model.fit(X, T, E)

In [240]:
model.model.net.eval()

X_np = np.asarray(X).astype("float32")

surv_f = model.model.predict_surv_df(X_np)


dh_expected_time = pd.Series(trapz(surv_f.T.values, surv_f.index), index=surv_f.T.index)

dh_expected_time

0       350.153704
1       337.546935
2       295.672012
3       290.470147
4       315.291187
           ...    
1146    343.135426
1147    283.278161
1148    343.990151
1149    363.056695
1150    331.392516
Length: 1151, dtype: float64

In [241]:
expected_time_error(T, E, dh_expected_time)

204.13984294012428

In [243]:
ranking_error(T, E, dh_expected_time)

353.43749999999966

In [242]:
(T <= dh_expected_time).sum() / len(dh_expected_time)

0.8349261511728931

## DATE model

In [244]:
# stdlib
from typing import Callable, List, Optional, Tuple

# third party
import numpy as np
import torch
from pydantic import validate_arguments
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# synthcity absolute
import synthcity.logger as log
from synthcity.utils.reproducibility import enable_reproducible_results
from synthcity.plugins.models.mlp import MLP

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class TimeEventGAN(nn.Module):
    @validate_arguments(config=dict(arbitrary_types_allowed=True))
    def __init__(
        self,
        n_features: int,
        n_units_latent: int,
        generator_n_layers_hidden: int = 2,
        generator_n_units_hidden: int = 250,
        generator_nonlin: str = "leaky_relu",
        generator_nonlin_out: Optional[List[Tuple[str, int]]] = None,
        generator_n_iter: int = 5000,
        generator_batch_norm: bool = False,
        generator_dropout: float = 0,
        generator_loss: Optional[Callable] = None,
        generator_lr: float = 2e-4,
        generator_weight_decay: float = 1e-3,
        generator_residual: bool = True,
        generator_opt_betas: tuple = (0.9, 0.999),
        discriminator_n_layers_hidden: int = 3,
        discriminator_n_units_hidden: int = 300,
        discriminator_nonlin: str = "leaky_relu",
        discriminator_n_iter: int = 1,
        discriminator_batch_norm: bool = False,
        discriminator_dropout: float = 0.1,
        discriminator_loss: Optional[Callable] = None,
        discriminator_lr: float = 2e-4,
        discriminator_weight_decay: float = 1e-3,
        discriminator_opt_betas: tuple = (0.9, 0.999),
        batch_size: int = 100,
        n_iter_print: int = 50,
        seed: int = 0,
        n_iter_min: int = 100,
        clipping_value: int = 0,
        lambda_gradient_penalty: float = 10,
    ) -> None:
        super(TimeEventGAN, self).__init__()

        self.n_features = n_features
        self.n_units_latent = n_units_latent
        self.lambda_gradient_penalty = lambda_gradient_penalty

        self.generator = MLP(
            task_type="regression",
            n_units_in=n_features + n_units_latent,
            n_units_out=1,  # time to event
            n_layers_hidden=generator_n_layers_hidden,
            n_units_hidden=generator_n_units_hidden,
            nonlin=generator_nonlin,
            nonlin_out=generator_nonlin_out,
            n_iter=generator_n_iter,
            batch_norm=generator_batch_norm,
            dropout=generator_dropout,
            loss=generator_loss,
            seed=seed,
            lr=generator_lr,
            residual=generator_residual,
            opt_betas=generator_opt_betas,
        ).to(DEVICE)

        self.discriminator = MLP(
            task_type="regression",
            n_units_in=n_features + 1,
            n_units_out=1,  # fake/true
            n_layers_hidden=discriminator_n_layers_hidden,
            n_units_hidden=discriminator_n_units_hidden,
            nonlin=discriminator_nonlin,
            nonlin_out=[("none", 1)],
            n_iter=discriminator_n_iter,
            batch_norm=discriminator_batch_norm,
            dropout=discriminator_dropout,
            loss=discriminator_loss,
            seed=seed,
            lr=discriminator_lr,
            opt_betas=discriminator_opt_betas,
        ).to(DEVICE)

        # training
        self.generator_n_iter = generator_n_iter
        self.discriminator_n_iter = discriminator_n_iter
        self.n_iter_print = n_iter_print
        self.n_iter_min = n_iter_min
        self.batch_size = batch_size
        self.clipping_value = clipping_value

        self.seed = seed
        enable_reproducible_results(seed)

    def fit(
        self,
        X: np.ndarray,
        T: np.ndarray,
        E: np.ndarray,
    ) -> "GAN":
        Xt = self._check_tensor(X)
        Tt = self._check_tensor(T)
        Et = self._check_tensor(E)

        self._train(
            Xt,
            Tt,
            Et,
        )

        return self

    def generate(self, X: np.ndarray) -> np.ndarray:
        X = self._check_tensor(X).float()

        return self(X).cpu().numpy()

    @validate_arguments(config=dict(arbitrary_types_allowed=True))
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        self.generator.eval()

        fixed_noise = torch.randn(len(X), self.n_units_latent, device=DEVICE)
        with torch.no_grad():
            return self.generator(torch.hstack([X, fixed_noise])).detach().cpu()

    def dataloader(
        self, X: torch.Tensor, T: torch.Tensor, E: torch.Tensor
    ) -> DataLoader:
        dataset = TensorDataset(X, T, E)

        return DataLoader(dataset, batch_size=self.batch_size, pin_memory=False)

    def _generate_training_outputs(
        self, X: torch.Tensor, T: torch.Tensor, E: torch.Tensor
    ) -> tuple:
        # Train with non-censored true batch
        Xnc = X[E == 1].to(DEVICE)
        Tnc = T[E == 1].to(DEVICE)
        true_labels = torch.ones((len(Xnc),), device=DEVICE)
        true_features_time = torch.hstack([Xnc, Tnc.reshape(-1, 1)])

        true_output = self.discriminator(true_features_time).squeeze().float()

        # Train with fake batch
        noise = torch.randn(len(X), self.n_units_latent, device=DEVICE)
        noise = torch.hstack([X, noise])
        fake_T = self.generator(noise)

        fake_labels = torch.zeros((len(X),), device=DEVICE)
        fake_features_time = torch.hstack([X, fake_T.reshape(-1, 1)])

        fake_output = self.discriminator(fake_features_time.detach()).squeeze()

        return true_features_time, true_output, fake_features_time, fake_output

    def _train_epoch_generator(
        self,
        X: torch.Tensor,
        T: torch.Tensor,
        E: torch.Tensor,
    ) -> float:
        # Update the G network
        self.generator.optimizer.zero_grad()

        # Evaluate noncensored error
        Xnc = X[E == 1].to(DEVICE)
        Tnc = T[E == 1].to(DEVICE)

        batch_size = len(Xnc)

        noise = torch.randn(batch_size, self.n_units_latent, device=DEVICE)
        noncen_input = torch.hstack([Xnc, noise])
        fake_T = self.generator(noncen_input)

        errG_noncen = nn.MSELoss()(
            fake_T, T
        )  # fake_T should be == T for noncensored data

        # Evaluate censored error
        Xc = X[E == 0].to(DEVICE)
        Tc = T[E == 0].to(DEVICE)

        batch_size = len(Xc)

        noise = torch.randn(batch_size, self.n_units_latent, device=DEVICE)
        cen_input = torch.hstack([Xc, noise])
        fake_T = self.generator(cen_input)

        errG_cen = torch.mean(
            nn.ReLU()(Tc - fake_T)
        )  # fake_T should be >= T for censored data

        # Discriminator loss
        _, real_output, _, fake_output = self._generate_training_outputs(X, T, E)

        errG_discr = torch.mean(real_output) - torch.mean(fake_output)
        # Calculate G's loss based on this output
        errG = errG_noncen + errG_cen + errG_discr

        assert errG.isnan().sum() == 0

        # Calculate gradients for G
        errG.backward()

        # Update G
        if self.clipping_value > 0:
            torch.nn.utils.clip_grad_norm_(
                self.generator.parameters(), self.clipping_value
            )
        self.generator.optimizer.step()

        # Return loss
        return errG.item()

    def _train_epoch_discriminator(
        self,
        X: torch.Tensor,
        T: torch.Tensor,
        E: torch.Tensor,
    ) -> float:
        # Update the D network
        errors = []

        batch_size = min(self.batch_size, len(X))

        for epoch in range(self.discriminator_n_iter):
            self.discriminator.zero_grad()

            (
                true_features_time,
                true_output,
                fake_features_time,
                fake_output,
            ) = self._generate_training_outputs(X, T, E)

            act = nn.Sigmoid()
            errD = -torch.mean(torch.log(act(true_output))) - torch.mean(
                torch.log(1 - act(fake_output))
            )

            assert errD.isnan().sum() == 0
            errD.backward()

            # Update D
            if self.clipping_value > 0:
                torch.nn.utils.clip_grad_norm_(
                    self.discriminator.parameters(), self.clipping_value
                )
            self.discriminator.optimizer.step()

            errors.append(errD.item())

        return np.mean(errors)

    def _train_epoch(
        self,
        loader: DataLoader,
    ) -> Tuple[float, float]:

        G_losses = []
        D_losses = []

        for i, data in enumerate(loader):
            G_losses.append(
                self._train_epoch_generator(
                    *data,
                )
            )
            D_losses.append(
                self._train_epoch_discriminator(
                    *data,
                )
            )

        return np.mean(G_losses), np.mean(D_losses)

    def _train(
        self,
        X: torch.Tensor,
        T: torch.Tensor,
        E: torch.Tensor,
    ) -> "TimeEventGAN":
        X = self._check_tensor(X).float()
        T = self._check_tensor(T).float()
        E = self._check_tensor(E).long()

        # Load Dataset
        loader = self.dataloader(X, T, E)

        # Train loop
        for i in range(self.generator_n_iter):
            g_loss, d_loss = self._train_epoch(loader)
            # Check how the generator is doing by saving G's output on fixed_noise
            if (i + 1) % self.n_iter_print == 0:
                print(
                    f"[{i}/{self.generator_n_iter}]\tLoss_D: {d_loss}\tLoss_G: {g_loss}"
                )

        return self

    def _check_tensor(self, X: torch.Tensor) -> torch.Tensor:
        if isinstance(X, torch.Tensor):
            return X.to(DEVICE)
        else:
            return torch.from_numpy(np.asarray(X)).to(DEVICE)

In [245]:
from sklearn.preprocessing import MinMaxScaler

model = TimeEventGAN(
    n_features=X.shape[1],
    n_units_latent=10,
)


scaler_X = MinMaxScaler()
enc_X = scaler_X.fit_transform(X)

scaler_T = MinMaxScaler()
enc_T = scaler_T.fit_transform(T.values.reshape(-1, 1)).squeeze()

model.fit(enc_X, enc_T, E)

[49/5000]	Loss_D: 0.20019801488767067	Loss_G: 11.527365763982138
[99/5000]	Loss_D: 0.12331527331843972	Loss_G: 14.093079487482706
[149/5000]	Loss_D: 0.07148932960505287	Loss_G: 15.957402308781942
[199/5000]	Loss_D: 0.09080297748247783	Loss_G: 17.300039450327557
[249/5000]	Loss_D: 0.10174048303936918	Loss_G: 19.185155232747395
[299/5000]	Loss_D: 0.06687433959450573	Loss_G: 20.655827442804974
[349/5000]	Loss_D: 0.07761384453624487	Loss_G: 21.532618681589764
[399/5000]	Loss_D: 0.06795037506769101	Loss_G: 21.988333702087402
[449/5000]	Loss_D: 0.06315301696304232	Loss_G: 23.156710465749104
[499/5000]	Loss_D: 0.05330320028588176	Loss_G: 24.417380174001057
[549/5000]	Loss_D: 0.03193874141046157	Loss_G: 25.697362740834553
[599/5000]	Loss_D: 0.044267649529501796	Loss_G: 26.166507403055828
[649/5000]	Loss_D: 0.04674430992842341	Loss_G: 26.074907302856445
[699/5000]	Loss_D: 0.026824829129812617	Loss_G: 27.09940528869629
[749/5000]	Loss_D: 0.03517821369071802	Loss_G: 28.588987509409588
[799/5000]	

TimeEventGAN(
  (generator): MLP(
    (model): Sequential(
      (0): ResidualLayer(
        (model): Sequential(
          (0): Linear(in_features=21, out_features=250, bias=True)
          (1): LeakyReLU(negative_slope=0.01)
        )
      )
      (1): ResidualLayer(
        (model): Sequential(
          (0): Linear(in_features=271, out_features=271, bias=True)
          (1): LeakyReLU(negative_slope=0.01)
        )
      )
      (2): Linear(in_features=542, out_features=1, bias=True)
    )
    (loss): MSELoss()
  )
  (discriminator): MLP(
    (model): Sequential(
      (0): LinearLayer(
        (model): Sequential(
          (0): Linear(in_features=12, out_features=300, bias=True)
          (1): LeakyReLU(negative_slope=0.01)
        )
      )
      (1): LinearLayer(
        (model): Sequential(
          (0): Dropout(p=0.1, inplace=False)
          (1): Linear(in_features=300, out_features=300, bias=True)
          (2): LeakyReLU(negative_slope=0.01)
        )
      )
      (2): 

In [246]:
model.eval()

enc_pred_T = model.generate(enc_X)

adv_time_to_event = scaler_T.inverse_transform(enc_pred_T).squeeze()

adv_time_to_event

array([314.0263 , 393.81577, 360.4408 , ..., 346.5795 , 443.94827,
       355.42078], dtype=float32)

In [247]:
adv_time_to_event[E == 1]

array([267.54825, 245.98071, 262.05115, 243.53607, 250.33655, 228.9197 ,
       257.41516, 269.35757, 286.9277 , 247.93243, 233.30562, 244.12299,
       273.93744, 252.66742, 226.18564, 247.0547 , 252.74051, 252.9439 ,
       244.32718, 228.18945, 268.00543, 284.67722, 279.92752, 254.25429,
       231.64702, 242.30023, 272.16202, 258.86584, 298.59848, 252.22182,
       276.31607, 308.05933, 318.5639 , 248.59091, 244.41428, 275.648  ,
       253.58578, 247.47708, 286.98837, 239.87407, 270.72705, 240.30164,
       241.15096, 261.2846 , 258.7759 , 237.62503, 246.4053 , 263.5498 ,
       231.86751, 219.23193, 279.80328, 252.33907, 230.61731, 242.9455 ,
       226.53604, 261.59644, 226.7726 , 238.3301 , 244.26926, 253.35579,
       239.2181 , 252.02248, 288.96683, 267.7728 , 230.7128 , 235.33846,
       280.22635, 240.31956, 241.65619, 222.62445, 250.12166, 256.84702,
       232.65787, 243.95787, 243.41786, 265.9128 , 278.2363 , 260.14536,
       260.04416, 251.6236 , 224.98022, 288.61652, 

In [248]:
expected_time_error(T, E, adv_time_to_event)

170.14575064261015

In [249]:
ranking_error(T, E, adv_time_to_event)

216.59374999999991

In [250]:
(T <= adv_time_to_event).sum() / len(adv_time_to_event)

0.8696785403996524

## NN prediction

In [83]:
# stdlib
from typing import Callable, List, Optional, Tuple

# third party
import numpy as np
import torch
from pydantic import validate_arguments
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# synthcity absolute
import synthcity.logger as log
from synthcity.utils.reproducibility import enable_reproducible_results
from synthcity.plugins.models.mlp import MLP

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class KaplanMeier(nn.Module):
    def __init__(self):
        super(KaplanMeier, self).__init__()

    def forward(self, T: torch.Tensor, E: torch.Tensor):
        """
        Estimages the Kaplan-Meier survival function
        parameters:
        - t: time steps
        - e: whether death occured at time step (1) or not (0)
        """
        T = self._check_tensor(T)
        E = self._check_tensor(E)

        Tnc = T[E == 1]

        Tnc_unique, nc_counts = torch.unique(T[E == 1], return_counts=True)
        T_unique, T_idxs, counts = torch.unique(
            T, return_inverse=True, return_counts=True
        )

        result = torch.ones(1, requires_grad=True).to(DEVICE)

        for i, t in enumerate(Tnc_unique):
            di = nc_counts[i]
            ni = torch.sum(counts[T_unique > t])

            result *= 1 - di / ni
        return result

    def _check_tensor(self, X: torch.Tensor) -> torch.Tensor:
        if isinstance(X, torch.Tensor):
            return X.to(DEVICE)
        else:
            return torch.from_numpy(np.asarray(X)).to(DEVICE)

In [84]:
T[E == 1].min()

1.0

In [85]:
model = KaplanMeier()

model(T, E)

tensor([0.9007], device='cuda:0', grad_fn=<MulBackward0>)

In [426]:
from typing import Callable

import pandas as pd
import torch
import torch.utils.data
import torchvision


class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset

    Arguments:
        indices: a list of indices
        num_samples: number of samples to draw
        callback_get_label: a callback-like function which takes two arguments - dataset and index
    """

    def __init__(self, X, T, E):
        # if indices is not provided, all elements in the dataset will be considered
        self.indices = list(range(X.shape[0]))

        # if num_samples is not provided, draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices)

        # distribution of classes in the dataset
        df = pd.DataFrame()
        df["label"] = E.cpu().numpy()
        df.index = self.indices
        df = df.sort_index()

        label_to_count = df["label"].value_counts()

        weights = 1.0 / label_to_count[df["label"]]

        self.weights = torch.DoubleTensor(weights.to_list())

    def __iter__(self):
        return (
            self.indices[i]
            for i in torch.multinomial(self.weights, self.num_samples, replacement=True)
        )

    def __len__(self):
        return self.num_samples

In [452]:
# stdlib
from typing import Callable, List, Optional, Tuple

# third party
import numpy as np
import torch
from pydantic import validate_arguments
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# synthcity absolute
import synthcity.logger as log
from synthcity.utils.reproducibility import enable_reproducible_results
from synthcity.plugins.models.mlp import MLP

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class TimeEventNN(nn.Module):
    @validate_arguments(config=dict(arbitrary_types_allowed=True))
    def __init__(
        self,
        n_features: int,
        n_layers_hidden: int = 2,
        n_units_hidden: int = 250,
        nonlin: str = "leaky_relu",
        nonlin_out: Optional[List[Tuple[str, int]]] = None,
        n_iter: int = 5000,
        batch_norm: bool = False,
        dropout: float = 0,
        lr: float = 2e-4,
        weight_decay: float = 1e-3,
        residual: bool = True,
        opt_betas: tuple = (0.9, 0.999),
        batch_size: int = 1000,
        n_iter_print: int = 500,
        seed: int = 0,
        n_iter_min: int = 100,
        clipping_value: int = 0,
        lambda_calibration=10,
        lambda_regression_nc=10,
        lambda_regression_c=1,
        verbose: bool = False,
    ) -> None:
        super(TimeEventNN, self).__init__()

        self.n_features = n_features
        self.lambda_calibration = lambda_calibration
        self.lambda_regression_nc = lambda_regression_nc
        self.lambda_regression_c = lambda_regression_c
        self.verbose = verbose

        self.generator = MLP(
            task_type="regression",
            n_units_in=n_features,
            n_units_out=1,  # time to event
            n_layers_hidden=n_layers_hidden,
            n_units_hidden=n_units_hidden,
            nonlin=nonlin,
            nonlin_out=nonlin_out,
            n_iter=n_iter,
            batch_norm=batch_norm,
            dropout=dropout,
            seed=seed,
            lr=lr,
            residual=residual,
            opt_betas=opt_betas,
        ).to(DEVICE)

        # training
        self.n_iter = n_iter
        self.n_iter_print = n_iter_print
        self.n_iter_min = n_iter_min
        self.batch_size = batch_size
        self.clipping_value = clipping_value

        self.seed = seed
        enable_reproducible_results(seed)

    def fit(
        self,
        X: np.ndarray,
        T: np.ndarray,
        E: np.ndarray,
    ) -> "TimeEventNN":
        Xt = self._check_tensor(X)
        Tt = self._check_tensor(T)
        Et = self._check_tensor(E)

        self._train(
            Xt,
            Tt,
            Et,
        )

        return self

    def generate(self, X: np.ndarray) -> np.ndarray:
        X = self._check_tensor(X).float()

        return self(X).cpu().numpy()

    @validate_arguments(config=dict(arbitrary_types_allowed=True))
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        self.generator.eval()

        with torch.no_grad():
            return self.generator(X).detach().cpu()

    def dataloader(
        self, X: torch.Tensor, T: torch.Tensor, E: torch.Tensor
    ) -> DataLoader:
        dataset = TensorDataset(X, T, E)
        sampler = ImbalancedDatasetSampler(X, T, E)

        return DataLoader(
            dataset, batch_size=self.batch_size, sampler=sampler, pin_memory=False
        )

    def _train_epoch_generator(
        self,
        X: torch.Tensor,
        T: torch.Tensor,
        E: torch.Tensor,
    ) -> float:
        # Update the G network
        self.generator.optimizer.zero_grad()

        # Calculate G's loss based on noncensored data
        errG_nc = self._loss_regression_nc(
            X[E == 1], T[E == 1]
        ) + self._loss_calibration(X[E == 1], T[E == 1])

        # Calculate G's loss based on censored data
        errG_c = self._loss_regression_c(X[E == 0], T[E == 0])

        # Calculate total loss
        errG = errG_nc + errG_c

        # Calculate gradients for G
        errG.backward()

        # Update G
        if self.clipping_value > 0:
            torch.nn.utils.clip_grad_norm_(
                self.generator.parameters(), self.clipping_value
            )
        self.generator.optimizer.step()

        # Return loss
        return errG.item()

    def _train_epoch(
        self,
        loader: DataLoader,
    ) -> float:

        G_losses = []

        for i, data in enumerate(loader):
            G_losses.append(
                self._train_epoch_generator(
                    *data,
                )
            )

        return np.mean(G_losses)

    def _train(
        self,
        X: torch.Tensor,
        T: torch.Tensor,
        E: torch.Tensor,
    ) -> "TimeEventGAN":
        X = self._check_tensor(X).float()
        T = self._check_tensor(T).float()
        E = self._check_tensor(E).long()

        # Load Dataset
        loader = self.dataloader(X, T, E)

        # Train loop
        for i in range(self.n_iter):
            g_loss = self._train_epoch(loader)
            # Check how the generator is doing by saving G's output on fixed_noise
            if self.verbose and (i + 1) % self.n_iter_print == 0:
                print(f"[{i}/{self.n_iter}]\tLoss_G: {g_loss}")

        return self

    def _check_tensor(self, X: torch.Tensor) -> torch.Tensor:
        if isinstance(X, torch.Tensor):
            return X.to(DEVICE)
        else:
            return torch.from_numpy(np.asarray(X)).to(DEVICE)

    def _loss_calibration(
        self,
        X: torch.Tensor,
        T: torch.Tensor,
    ) -> float:
        # Evaluate noncensored error
        X = X.to(DEVICE)
        T = T.to(DEVICE)
        pred_T = self.generator(X).squeeze()

        def _inner_dist(arr):
            lhs = arr.view(-1, 1).repeat(1, len(arr))

            return lhs - lhs.T

        inner_T_dist = _inner_dist(T)
        inner_pred_T_dist = _inner_dist(pred_T)

        return self.lambda_calibration * nn.MSELoss()(inner_T_dist, inner_pred_T_dist)

    def _loss_regression_c(
        self,
        X: torch.Tensor,
        T: torch.Tensor,
    ) -> float:
        # Evaluate censored error
        X = X.to(DEVICE)
        T = T.to(DEVICE)
        fake_T = self.generator(X)

        errG_cen = torch.mean(
            nn.ReLU()(T - fake_T)
        )  # fake_T should be >= T for censored data

        # Calculate G's loss based on this output
        return self.lambda_regression_c * errG_cen

    def _loss_regression_nc(
        self,
        X: torch.Tensor,
        T: torch.Tensor,
    ) -> float:
        # Evaluate noncensored error
        X = X.to(DEVICE)
        T = T.to(DEVICE)
        fake_T = self.generator(X)

        errG_noncen = nn.MSELoss()(
            fake_T, T
        )  # fake_T should be == T for noncensored data

        return self.lambda_regression_nc * errG_noncen

In [464]:
from sklearn.preprocessing import MinMaxScaler

args = {
    "n_layers_hidden": 5,
    "n_units_hidden": 300,
    "nonlin": "leaky_relu",
    "batch_norm": False,
    "dropout": 0.061771005405779913,
    "lr": 0.001,
    "residual": False,
    "batch_size": 250,
    "lambda_calibration": 10,
    "lambda_regression_nc": 50,
    "lambda_regression_c": 100,
}

model = TimeEventNN(
    n_features=X.shape[1],
    verbose=True,
    n_iter=7000,
    **args,
)


scaler_X = MinMaxScaler()
enc_X = scaler_X.fit_transform(X)

scaler_T = MinMaxScaler()
enc_T = scaler_T.fit_transform(T.values.reshape(-1, 1)).squeeze()

_ = model.fit(enc_X, enc_T, E)

[499/7000]	Loss_G: 3.3376758098602295
[999/7000]	Loss_G: 2.5953210830688476
[1499/7000]	Loss_G: 3.282554006576538
[1999/7000]	Loss_G: 3.105288362503052
[2499/7000]	Loss_G: 3.2453786849975588
[2999/7000]	Loss_G: 2.88686466217041
[3499/7000]	Loss_G: 3.3921277046203615
[3999/7000]	Loss_G: 3.1902450561523437
[4499/7000]	Loss_G: 3.3084179878234865
[4999/7000]	Loss_G: 2.8367167949676513
[5499/7000]	Loss_G: 2.9377728939056396
[5999/7000]	Loss_G: 3.0153308391571043
[6499/7000]	Loss_G: 2.8836273193359374
[6999/7000]	Loss_G: 2.9327690601348877


In [465]:
model.eval()

enc_pred_T = model.generate(enc_X)

nn_time_to_event = scaler_T.inverse_transform(enc_pred_T).squeeze()

nn_time_to_event[E == 1]

array([135.50903 , 156.26138 , 130.72322 , 102.08743 ,  89.81493 ,
       109.457184,  77.09829 , 107.370026, 126.5123  ,  94.76485 ,
        81.10462 ,  97.82861 ,  78.832436, 124.84132 ,  77.28345 ,
        84.33965 , 106.36029 ,  99.56727 , 162.57362 ,  81.21036 ,
       103.52593 ,  81.37758 , 106.711044,  92.11203 , 115.14865 ,
        85.97696 , 101.89047 , 109.74017 , 143.50134 , 103.52821 ,
       100.02467 ,  77.67313 , 101.65781 , 145.27959 ,  93.36886 ,
        95.60155 , 101.93185 , 102.141396,  82.05487 ,  77.1862  ,
       159.59782 , 112.75272 ,  89.26897 , 111.00989 ,  76.352295,
        84.64619 , 103.18254 ,  77.86974 , 111.73801 ,  92.1155  ,
        87.00022 , 141.66734 , 140.58092 ,  79.11524 ,  78.059135,
        96.59738 , 102.839584,  94.07241 ,  83.65452 ,  93.17826 ,
        74.23955 , 143.58727 ,  75.559654, 122.65561 ,  82.35041 ,
        80.23777 , 114.29236 , 131.49608 ,  93.977905,  75.604294,
        82.44831 , 104.547455,  99.078705,  79.57662 ,  77.168

In [466]:
expected_time_error(T, E, nn_time_to_event)

54.56784196623553

In [467]:
ranking_error(T, E, nn_time_to_event)

133.76041666666669

In [468]:
(T <= nn_time_to_event).sum() / len(nn_time_to_event)

0.9574283231972198

In [463]:
import optuna


optuna.logging.set_verbosity(optuna.logging.DEBUG)
optuna.logging.enable_propagation()
optuna.logging.enable_default_handler()


def _trial_params(trial, param_space):
    out = {}

    for param in param_space:
        if hasattr(param, "choices"):
            out[param.name] = trial.suggest_categorical(
                param.name, choices=param.choices
            )
        elif hasattr(param, "step"):
            out[param.name] = trial.suggest_int(
                param.name, param.low, param.high, param.step
            )
        else:
            out[param.name] = trial.suggest_float(param.name, param.low, param.high)

    return out


def objective(trial):
    args = {
        "n_layers_hidden": trial.suggest_int("n_layers_hidden", 1, 5),
        "n_units_hidden": trial.suggest_int("n_units_hidden", 50, 300, step=50),
        "nonlin": trial.suggest_categorical("nonlin", ["relu", "leaky_relu"]),
        "n_iter": 500,
        "batch_norm": trial.suggest_categorical("batch_norm", [True, False]),
        "dropout": trial.suggest_float("dropout", 0, 0.2),
        "lr": trial.suggest_categorical("lr", [1e-2, 1e-3, 1e-4]),
        "residual": trial.suggest_categorical("residual", [True, False]),
        "batch_size": trial.suggest_categorical("batch_size", [100, 250, 500, 1000]),
        "lambda_calibration": trial.suggest_categorical(
            "lambda_calibration", [1, 10, 50, 100]
        ),
        "lambda_regression_nc": trial.suggest_categorical(
            "lambda_regression_nc", [1, 10, 50, 100]
        ),
        "lambda_regression_c": trial.suggest_categorical(
            "lambda_regression_c", [1, 10, 50, 100]
        ),
    }

    model = TimeEventNN(
        n_features=X.shape[1],
        **args,
    )
    model.fit(enc_X, enc_T, E)
    model.eval()

    enc_pred_T = model.generate(enc_X)

    nn_time_to_event = scaler_T.inverse_transform(enc_pred_T).squeeze()

    return ranking_error(T, E, nn_time_to_event)


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

[I 2022-04-27 12:50:13,141] A new study created in memory with name: no-name-6f00874f-ae6b-41aa-b635-dd2e29b8c49c
[I 2022-04-27 12:51:52,024] Trial 0 finished with value: 213.61458333333343 and parameters: {'n_layers_hidden': 5, 'n_units_hidden': 250, 'nonlin': 'leaky_relu', 'batch_norm': False, 'dropout': 0.18158716416518192, 'lr': 0.001, 'residual': True, 'batch_size': 250, 'lambda_calibration': 100, 'lambda_regression_nc': 1, 'lambda_regression_c': 100}. Best is trial 0 with value: 213.61458333333343.
[I 2022-04-27 12:52:10,914] Trial 1 finished with value: 318.7291666666671 and parameters: {'n_layers_hidden': 2, 'n_units_hidden': 250, 'nonlin': 'relu', 'batch_norm': True, 'dropout': 0.03171068850288972, 'lr': 0.001, 'residual': True, 'batch_size': 250, 'lambda_calibration': 10, 'lambda_regression_nc': 10, 'lambda_regression_c': 100}. Best is trial 0 with value: 213.61458333333343.
[I 2022-04-27 12:52:27,938] Trial 2 finished with value: 364.3645833333329 and parameters: {'n_layers_

[I 2022-04-27 12:59:16,587] Trial 21 finished with value: 142.09375000000003 and parameters: {'n_layers_hidden': 4, 'n_units_hidden': 300, 'nonlin': 'leaky_relu', 'batch_norm': False, 'dropout': 0.10469494723478397, 'lr': 0.001, 'residual': False, 'batch_size': 250, 'lambda_calibration': 10, 'lambda_regression_nc': 50, 'lambda_regression_c': 100}. Best is trial 21 with value: 142.09375000000003.
[I 2022-04-27 12:59:39,241] Trial 22 finished with value: 142.5000000000001 and parameters: {'n_layers_hidden': 5, 'n_units_hidden': 300, 'nonlin': 'leaky_relu', 'batch_norm': False, 'dropout': 0.05605696899266862, 'lr': 0.001, 'residual': False, 'batch_size': 250, 'lambda_calibration': 10, 'lambda_regression_nc': 50, 'lambda_regression_c': 100}. Best is trial 21 with value: 142.09375000000003.
[I 2022-04-27 13:00:01,881] Trial 23 finished with value: 139.93750000000006 and parameters: {'n_layers_hidden': 5, 'n_units_hidden': 300, 'nonlin': 'leaky_relu', 'batch_norm': False, 'dropout': 0.061771

[I 2022-04-27 13:07:04,880] Trial 41 finished with value: 144.77083333333337 and parameters: {'n_layers_hidden': 5, 'n_units_hidden': 300, 'nonlin': 'leaky_relu', 'batch_norm': False, 'dropout': 0.020316355297418716, 'lr': 0.001, 'residual': False, 'batch_size': 250, 'lambda_calibration': 10, 'lambda_regression_nc': 50, 'lambda_regression_c': 100}. Best is trial 23 with value: 139.93750000000006.
[I 2022-04-27 13:07:28,801] Trial 42 finished with value: 140.17708333333326 and parameters: {'n_layers_hidden': 5, 'n_units_hidden': 300, 'nonlin': 'leaky_relu', 'batch_norm': False, 'dropout': 0.0621653207199051, 'lr': 0.001, 'residual': False, 'batch_size': 250, 'lambda_calibration': 10, 'lambda_regression_nc': 50, 'lambda_regression_c': 100}. Best is trial 23 with value: 139.93750000000006.
[I 2022-04-27 13:07:57,270] Trial 43 finished with value: 145.33333333333337 and parameters: {'n_layers_hidden': 5, 'n_units_hidden': 250, 'nonlin': 'leaky_relu', 'batch_norm': False, 'dropout': 0.07698

## Predict censoring time, as seen in the dataset

In [33]:
synthetic_expected_time = pd.Series(adv_time_to_event, index=X.index)

In [34]:
# Predict censoring time
pred_T = pd.Series(synthetic_expected_time, index=X.index)

fill_X = X[E == 0].copy()
fill_X["pred_T"] = pred_T[E == 0]

fill_X

,age,cd4,hemophil,ivdrug,karnof,priorzdv,raceth,sex,strat2,tx,txgrp,pred_T
0,34.0,169.0,0,0,0,39.0,0,0,1,0,0,1.000000
1,34.0,149.5,0,0,3,15.0,1,1,1,0,0,1.000000
2,20.0,23.5,1,0,0,9.0,0,0,0,1,1,1.000004
3,48.0,46.0,0,0,3,53.0,0,0,1,0,0,1.000000
4,46.0,10.0,0,2,3,12.0,0,0,0,1,1,1.000001
...,...,...,...,...,...,...,...,...,...,...,...,...
1145,25.0,16.5,0,0,0,7.0,0,0,0,0,0,1.000001
1146,44.0,65.5,0,0,0,103.0,0,0,1,1,1,1.000000
1148,43.0,170.0,0,2,3,27.0,1,0,1,0,0,1.000000
1149,44.0,282.5,0,2,2,12.0,0,0,1,0,0,1.000000


In [35]:
from adjutorium.plugins.prediction.regression import Regression

xgb_regression = Regression().get("xgboost_regressor")

xgb_regression.fit(fill_X, T[E == 0])

In [36]:
xgb_regression.predict(fill_X)

,0
0,210.753510
1,275.778442
2,245.945831
3,200.979675
4,301.189606
...,...
1050,294.570587
1051,275.296448
1052,269.980957
1053,183.680069


In [37]:
cens_samples = X[E == 0]

censoring_input = cens_samples.copy()
censoring_input["pred_T"] = synthetic_expected_time[cens_samples.index]

censoring_input

,age,cd4,hemophil,ivdrug,karnof,priorzdv,raceth,sex,strat2,tx,txgrp,pred_T
0,34.0,169.0,0,0,0,39.0,0,0,1,0,0,1.000000
1,34.0,149.5,0,0,3,15.0,1,1,1,0,0,1.000000
2,20.0,23.5,1,0,0,9.0,0,0,0,1,1,1.000004
3,48.0,46.0,0,0,3,53.0,0,0,1,0,0,1.000000
4,46.0,10.0,0,2,3,12.0,0,0,0,1,1,1.000001
...,...,...,...,...,...,...,...,...,...,...,...,...
1145,25.0,16.5,0,0,0,7.0,0,0,0,0,0,1.000001
1146,44.0,65.5,0,0,0,103.0,0,0,1,1,1,1.000000
1148,43.0,170.0,0,2,3,27.0,1,0,1,0,0,1.000000
1149,44.0,282.5,0,2,2,12.0,0,0,1,0,0,1.000000


In [38]:
synth_T = T
synth_T[cens_samples.index] = xgb_regression.predict(censoring_input).values.squeeze()

synth_E = pd.Series([1] * len(X), index=X.index)
synth_E[cens_samples.index] = 0

synth_E

0       0
1       0
2       0
3       0
4       0
       ..
1146    0
1147    1
1148    0
1149    0
1150    0
Length: 1151, dtype: int64

In [39]:
synth_T[synth_T <= 0] = 1

synth_T

0       210.753510
1       275.778442
2       245.945831
3       200.979675
4       301.189606
           ...    
1146    275.296448
1147     47.000000
1148    269.980957
1149    183.680069
1150    201.239227
Name: duration, Length: 1151, dtype: float64

In [40]:
# Baseline evaluation
from synthcity.plugins import Plugins
from adjutorium.utils.tester import evaluate_survival_estimator
from adjutorium.plugins.prediction.risk_estimation import RiskEstimation

predictor = RiskEstimation().get_type("survival_xgboost")
n_folds = 3

const_cols = constant_columns(X)
X = X.drop(columns=const_cols)

evaluate_survival_estimator(
    predictor(),
    X,
    T,
    E,
    time_horizons=time_horizons,
    n_folds=n_folds,
)["str"]

{'c_index': '0.7393 +/- 0.0234',
 'brier_score': '0.0616 +/- 0.0024',
 'aucroc': '0.7268 +/- 0.0062'}

In [41]:
# Baseline evaluation
from synthcity.plugins import Plugins
from adjutorium.utils.tester import evaluate_survival_estimator
from adjutorium.plugins.prediction.risk_estimation import RiskEstimation
from sklearn.model_selection import StratifiedKFold
from adjutorium.utils.metrics import (
    evaluate_skurv_brier_score,
    evaluate_skurv_c_index,
)
from adjutorium.utils.third_party.nonparametric import (
    CensoringDistributionEstimator,
    SurvivalFunctionEstimator,
)

predictor = RiskEstimation().get_type("survival_xgboost")
n_folds = 3

const_cols = constant_columns(X)
X = X.drop(columns=const_cols)


skf = StratifiedKFold()

time_horizons = np.linspace(synth_T.min(), synth_T.max(), num=5)[2:-1]

eval_X = X
eval_T = synth_T
eval_E = synth_E

for train_index, test_index in skf.split(eval_X, eval_E):
    X_train = eval_X.loc[eval_X.index[train_index]]
    Y_train = eval_E.loc[eval_E.index[train_index]]
    T_train = eval_T.loc[eval_T.index[train_index]]
    X_test = eval_X.loc[eval_X.index[test_index]]
    Y_test = eval_E.loc[eval_E.index[test_index]]
    T_test = eval_T.loc[eval_T.index[test_index]]

    model = predictor()
    model.fit(X_train, T_train, Y_train)

    pred = model.predict(X_test, time_horizons).values

    cest = CensoringDistributionEstimator()

    train_structured = [(Y_train.iloc[i], T_train.iloc[i]) for i in range(len(Y_train))]
    train_structured = np.array(
        train_structured, dtype=[("status", "bool"), ("time", "<f8")]
    )

    cest.fit(train_structured)

    for k in range(len(time_horizons)):
        eval_horizon = min(time_horizons[k], np.max(T_test) - 1)

        Y_filter = Y_test[T_test < eval_horizon]
        T_filter = T_test[T_test < eval_horizon]

        test_structured = [
            (Y_filter.iloc[i], T_filter.iloc[i]) for i in range(len(Y_filter))
        ]
        test_structured = np.array(
            test_structured, dtype=[("status", "bool"), ("time", "<f8")]
        )

        cest.predict_ipcw(test_structured)

        print(Y_test.sum(), Y_train.sum())
        score = evaluate_skurv_c_index(
            T_train,
            Y_train,
            pred[:, k],
            T_test,
            Y_test,
            eval_horizon,
        )
        print(score)

20 76
0.8058526240040862
20 76
0.7735097732622199
19 77
0.6938282076696742
19 77
0.6864121889174984
19 77
0.8058270948396166
19 77
0.7898577979116185
19 77
0.7424563520782076
19 77
0.7464877060572261
19 77
0.713249877748108
19 77
0.6398226815366069


In [42]:
pred_T[E == 1]

13      1.000001
16      1.000001
22      1.000000
29      1.000000
56      1.000000
          ...   
1075    1.000000
1106    1.000000
1120    1.000000
1126    1.000000
1147    1.000000
Length: 96, dtype: float32

In [43]:
T[E == 1]

13      206.0
16      298.0
22      190.0
29       82.0
56       61.0
        ...  
1075    186.0
1106    114.0
1120     20.0
1126    174.0
1147     47.0
Name: duration, Length: 96, dtype: float64

In [44]:
xgb_expected_time[E == 1]

13      262.474997
16      266.742325
22      273.714523
29      266.343808
56      266.526362
           ...    
1075    270.930636
1106    277.411832
1120    275.439517
1126    269.017622
1147    260.382612
Length: 96, dtype: float64